<b><font color="black" size="+3">Цветовые палитры видео</font></b>
<p>Программа раскладывает кадры видео на последовательность цветных полос</p>
<b><font color="black" size="+2">Сделано под впечатлением от :</font></b>

**Visualizations of movies in one figure**: [Пост на Reddit](https://www.reddit.com/r/dataisbeautiful/comments/glf9np/oc_visualizations_of_more_movies_in_one_figure/)


**Код и колаб:** [@akolomatskiy](https://t.me/akolomatskiy)

In [1]:
# import libraries
import os
import glob
import logging
import yadisk
from datetime import datetime
import time
from funcadd import save_data, load_data, update_progress
from processing import plot_lines, plot_donut, plot_interpolated_donut, plot_waves, gen_smooth_lines, gen_interpolated_lines
from postergenerator import create_poster

In [2]:
# enable logging
logfile = 'logs/{}.log'.format(datetime.now().strftime("%Y-%m-%dT%H:%M"))
logging.basicConfig(filename=logfile, filemode="w",
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

logger = logging.getLogger(__name__)

In [3]:
# connecting to Ya.Disk
logger.info('Connecting to YaDisk')
try:
    disk = yadisk.YaDisk(token = os.environ['YaDiskToken'])
    logger.info('Connected to YaDisk successfully')
except Exception as e:
    logger.error('Connecting to YaDisk failed with error %s. Quitting...', e)

In [4]:
# getting unprocessed files from Ya.Disk
logger.info('Getting unprocessed files from Ya.Disk')
try:
    del_datasphere_variables('ls')
except:
    pass

try:
    with open('progress.txt', 'r') as file:
        progress = [s.strip() for s in file.readlines()]
    ls = list(disk.listdir('MovieLines/raw'))
    raw_files = [film['path'].replace('disk:', '') for film in ls]
    files = sorted(list(set(raw_files) - set(progress)))
    print('\n'.join(files))
    logger.info('Got %s files to process from Ya.Disk', len(files))
except Exception as e:
    logger.error('Got error %s while getting files. Quitting...', e)

/MovieLines/raw/3Idiots_423210.mkv
/MovieLines/raw/Amelie_341.mkv
/MovieLines/raw/Avatar_251733.mkv
/MovieLines/raw/AvengersInfinityWar_843649.mkv
/MovieLines/raw/BackToTheFuture_476.mkv
/MovieLines/raw/Inception_447301.mkv
/MovieLines/raw/Joker_1048334.mkv
/MovieLines/raw/Madagascar_6006.mp4
/MovieLines/raw/MostersUniversity_468904.mkv
/MovieLines/raw/OnceUponATimeInHollywood_1047883.mkv
/MovieLines/raw/Rio_465674.mkv
/MovieLines/raw/SpiderMan_838.mkv
/MovieLines/raw/TheDarkKnight_111543.mkv
/MovieLines/raw/TheFifthElement_2656.mkv
/MovieLines/raw/TheMatrix_301.mkv
/MovieLines/raw/TheShawshankRedemption_326.mkv
/MovieLines/raw/Wall-E_279102.mkv
/MovieLines/raw/Zootopia_775276.mkv


In [ ]:
# ONLY FOR TESTS
# files = ['test/TheShawshankRedemption_326.mkv']
# files

In [5]:
# setting up parameters
# width, height, pix_per_line = 4096, 2048, 1
createSmoothDonutPoster = True
createInterpolatedDonutPoster = True
createSmoothWavesPoster = False

In [ ]:
# main loop
for video_file in files:
    start_time = time.time()
    name = video_file.split('.')[0].split('/')[-1]
    os.makedirs('output/{}'.format(name), exist_ok=True)
    output_folder = 'output/{}'.format(name)
    logger.info('Processing %s started', name)
   
    #checking files on drive
    saved_data = glob.glob('{}/*.pkl'.format(output_folder))
    video_path = os.path.join(output_folder, video_file.split('/')[-1])
    saved_data_exists = any('interpolatedLines' in string for string in saved_data) & any('smoothLines' in string for string in saved_data)
    if not saved_data_exists:
        # downloading file from YaDisk
        if not os.path.isfile(video_path):
            try:
                logger.info('Downloading %s started', name)
                disk.download(video_file, video_path)
                logger.info('Downloading %s is complete', name)
            except Exception as e:
                logger.error('Downloading raised error %s. Skipping %s', e, name)
                break
        
    # ONLY FOR TESTS
#     video_path = video_file
    
    # generating smooth lines
    logger.info('Getting smooth lines data')
    line_type = 'smoothLines' 
    sl_filename = '{}/{}_{}'.format(output_folder, name, line_type)
    try:
        smooth_lines = load_data(sl_filename)
        logger.info('Smooth lines data loaded')
    except FileNotFoundError:
        logger.info('No saved data. Generating smooth lines data. It may take some time...')
        try:
            smooth_lines = gen_smooth_lines(video_path)
            save_data(smooth_lines, sl_filename)
            logger.info('Smooth lines data generated successfully')
        except Exception as e:
            logger.warning('Generating smooth lines data raised error %s. Smooth lines data was not generated', e)
    
    # generating interpolated lines
    logger.info('Getting interpolated lines data')
    line_type = 'interpolatedLines'
    il_filename = '{}/{}_{}'.format(output_folder, name, line_type)
    try:
        interpolated_lines = load_data(il_filename)
        logger.info('Interpolated lines data loaded')
    except FileNotFoundError:
        logger.info('No saved data. Generating interpolated lines data. It may take some time...')
        try:
            interpolated_lines = gen_interpolated_lines(video_path)
            save_data(interpolated_lines, il_filename)
            logger.info('Interpolated lines data generated successfully')
        except Exception as e:
            logger.warning('Generating interpolated lines data raised error %s. Interpolated lines data was not generated', e)
    
    # generating donut data from smooth lines
    logger.info('Getting smooth donut data')
    line_type = 'smoothDonut'
    dsl_filename = '{}/{}_{}'.format(output_folder, name, line_type)
    try:
        pielines = smooth_lines[0]
    except FileNotFoundError:
        logger.warning('Getting smooth donut data raised error %s. Smooth donut data was not generated', e)
    
    if createSmoothDonutPoster:
        try:
            donut_path = dsl_filename + '.png'
            plot_donut(pielines, donut_path, background_style='beige')
            stripe_path = sl_filename + '.png'
            plot_lines(smooth_lines, stripe_path)
            output_path = dsl_filename + '_rendered_donut.png'
            create_poster(output_path, name, form='donut', donut_path=donut_path, stripe_path=stripe_path)
        except Exception as e:
            logger.error('Creating smooth donut data raised error %s. Smooth donut was not generated', e)
        
    if createInterpolatedDonutPoster:
        try:
            line_type = 'interpolatedDonut'
            dil_filename = '{}/{}_{}'.format(output_folder, name, line_type)
            interpolated_image_path = il_filename + '.png'
            plot_lines(interpolated_lines, interpolated_image_path)
            interdonut_path = dil_filename + '.png'
            plot_interpolated_donut(interpolated_image_path, interdonut_path, background_style='beige')
            output_path = dil_filename + '_rendered_interdonut.png'
            create_poster(output_path, name, form='donut', donut_path=interdonut_path, stripe_path=interpolated_image_path)
        except Exception as e:
            logger.error('Creating interpolated donut data raised error %s. Interpolated donut was not generated', e)
        
    # generating waves data from smooth lines
#     logger.info('Getting smooth wave data')
#     line_type = 'waveSmoothLines'
#     wsl_filename = '{}/{}_{}_{}x{}x{}'.format(output_folder, name, line_type, width, height, pix_per_line)
#     try:
#         wavelines = load_data(wsl_filename)
#         logger.info('Smooth wave data loaded')
#     except FileNotFoundError:
#         logger.info('No saved data. Generating smooth wave data')
#         try:
#             wavelines = smooth_lines[0]
#             save_data(pielines, wsl_filename)
#             logger.info('Smooth wave data generated successfully')
#         except Exception as e:
#             logger.warn('Generating smooth wave data raised error %s. Smooth wave data was not generated', e)
    
    logger.info('Posters creation for %s finished in %s seconds', name, round(time.time() - start_time, 2))
    
    logger.info('Uploading posters to Ya.Disk')
    try:
        posters = glob.glob('{}/*rendered*.png'.format(output_folder))
        for poster in posters:
            disk.upload(poster, 'MovieLines/output/images/{}'.format(poster.split('/')[-1]))
        logger.info('Uploading diagrams to Ya.Disk finished')
    except Exception as e:
        logger.error('Failed to upload posters to Ya.Disk: ', e)
                
#     logger.info('Uploading lines to Ya.Disk')
#     try: 
#         prepared_data = glob.glob('{}/*.pkl'.format(output_folder))
#         for lines in prepared_data:
#             disk.upload(lines, 'MovieLines/output/lines/{}'.format(lines.split('/')[-1]))
#             logger.info('Uploading lines to Ya.Disk finished')
#     except Exception as e:
#         logger.error('Failed to upload lines to Ya.Disk: ', e)
    
    logger.info('Deleting raw video file and images to save space')
    try:
        images = glob.glob('{}/.png'.format(output_folder))
        for image in images:
            os.remove(image)
        logger.info('Images deleted successfully')
        os.remove(video_path)
        logger.info('Video file deleted successfully')
    except Exception as e:
        logger.error('Failed to delete video file: ', e)
    
    logger.info('Overall processing for %s finished in %s seconds', name, round(time.time() - start_time, 2))
    
    update_progress(video_file)

  6%|▌         | 583/10267 [00:33<11:52, 13.59it/s]